<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENT_ORCHESTRARION_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Mar 22 20:03:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   47C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade crewai langchain -q
!pip install --upgrade colab-env --quiet
!pip install --upgrade langchain_community -q
!pip install --upgrade faiss-cpu -q
!pip install --upgrade crewai-tools -q
!pip install 'crewai[tools]'
!pip install transformers -U -q

In [2]:
import warnings

warnings.filterwarnings("ignore", message="You seem to be using the pipelines sequentially on GPU")

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive


In [3]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.llms import HuggingFaceHub

In [4]:
import colab_env
import os

In [5]:
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [11]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from litellm import completion  # Import litellm directly

load_dotenv()

# 1. Token Management
access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")
if not access_token_write:
    access_token_write = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not access_token_write:
    access_token_write = input("Enter your Hugging Face API token: ")

if not access_token_write:
    raise ValueError("Hugging Face API token is not set.")

os.environ["MODEL_PROVIDER_HUGGINGFACE"] = "true"
os.environ["MODEL_PROVIDER"] = "huggingface"
os.environ["HUGGINGFACE_API_KEY"] = access_token_write  # Set Hugging Face API key for litellm

print("Hugging Face access token is ready.")
print('\n')

# 2. Initialize LLM (No need for crewai.llm.LLM or HuggingFacePipeline here)
model_name = "huggingface/meta-llama/Meta-Llama-3.1-8B-Instruct"  # Use full Hugging Face path

# 3. Define Agents (Modify agent behavior to use litellm.completion)
def agent_run_with_litellm(self, task_description):
    """
    This function replaces the default agent run behavior to use litellm.completion.
    """
    messages = [{"role": "user", "content": task_description}]
    response = completion(model=model_name, messages=messages)  # Use litellm.completion
    return response.choices[0].message.content  # Extract content from response

# Monkey patch the agent's run method
Agent.run = agent_run_with_litellm

# 4. Define Agents
researcher = Agent(
    role='Senior Research Analyst',
    goal='Conduct thorough research on a given topic using only the provided information.',
    backstory="You are a seasoned research analyst with a knack for finding relevant information and synthesizing it into concise summaries.",
    allow_delegation=False,
    verbose=True,
)

writer = Agent(
    role='Technical Writer',
    goal='Write a clear and concise tutorial based on research using only the provided information.',
    backstory="You are an expert technical writer, skilled in simplifying complex topics into easy-to-understand tutorials.",
    allow_delegation=False,
    verbose=True,
)

# 5. Define Tasks
research_task = Task(
    description="Research the basics of crewAI, focusing on agents, tasks, and crew concepts. Provide a concise summary of each concept and how they relate to each other.",
    agent=researcher,
    expected_output="A concise summary of crewAI, agents, tasks, and crew concepts, highlighting their relationships.",
)

write_tutorial_task = Task(
    description="Write a short tutorial explaining crewAI using the research provided by the researcher agent. Focus on clarity and simplicity for a beginner audience. Include code examples if relevant.",
    agent=writer,
    expected_output="A short tutorial explaining crewAI concepts in a way that is easy for beginners to understand. Include code examples if relevant.",
)

# 6. Create and Run the Crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_tutorial_task],
    process=Process.sequential
)

try:
    result = crew.kickoff()
    print("Crew's Output:")
    print(result)

except Exception as e:
    print(f"An error occurred during crew execution: {e}")

Hugging Face access token is ready.


# Agent: Senior Research Analyst
## Task: Research the basics of crewAI, focusing on agents, tasks, and crew concepts. Provide a concise summary of each concept and how they relate to each other.


# Agent: Senior Research Analyst
## Final Answer: 
**CrewAI Overview**  
CrewAI is an advanced artificial intelligence system designed to optimize and manage various tasks through the collaborative efforts of software agents. This framework is centered around three key concepts: agents, tasks, and crews. Each of these elements plays a crucial role in the operation and efficiency of the CrewAI system.

**1. Agents**  
Agents are the autonomous software entities that operate within the CrewAI framework. They are designed to perform specific roles or functions based on predefined algorithms or learning models. Each agent can analyze data, make decisions, and execute tasks without requiring constant human oversight. Agents can be specialized, focusing on par